In [1]:
# Libraries Needed
import os
import re
import sys
import time
from datetime import datetime
import pandas as pd
import numpy as np
from typing import Any, List, Tuple, Union
import sklearn
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import pickle
from tqdm import tqdm

In [62]:
# create dataset start
def create_dataset(df):
    
    X_dataset = []
    img_list = df['Path'].tolist()
    for i in tqdm(range(df.shape[0])):
        
        img = image.load_img(img_list[i])
        img = image.img_to_array(img)
        img = img / 255.
        X_dataset.append(img)

    X = np.array(X_dataset)
  # drop unnecessary columns not used for training
    y = np.array(df.drop(['Index', 'Filename','Path','Emotion'], axis=1))

    return X, y
# create dataset end
def label_binarizer(y_intensity,threshold_val):
    return np.where(y_intensity<threshold_val, 0, 1)
def subset_accuracy(y_true,y_pred):
    y_true = tensorflow.py_function(label_binarizer,(y_true,0.5), tensorflow.double)
    y_pred = tensorflow.py_function(label_binarizer,(y_pred,0.5), tensorflow.double)
    return tensorflow.py_function(accuracy_score,(y_true,y_pred),tensorflow.double)

def load_model_data(model_path, opt_path):
    model = load_model(model_path,custom_objects={"subset_accuracy":subset_accuracy})
    return model
# define Assume Negative loss
def loss_an(y_pred, y_true):
    bce_loss =tensorflow.keras.losses.BinaryCrossentropy()
    return bce_loss(y_pred,y_true)

# create model start
def create_model(width=224, height=224):
  # load pretrained model 'VGG16'
    base_model = keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(width, height, 3))
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    
    model.add(Dense(12, activation='sigmoid', name='final_au', kernel_initializer='glorot_normal'))
    
  # sigmoid classification for 12 au labels
    model.compile(optimizer='adam', loss=loss_an, metrics=[subset_accuracy])
    return model
# create model end


# create train_ER_model
# def train_AU_model(X_train, y_train, model):
#   start_time = time.time()
#   EPOCHS = 20
#   kfold = KFold(3, shuffle=True, random_state=123)
#   for f, (trn_ind, val_ind) in enumerate(kfold.split(X_train)):
#     print();
#     print('#' * 50)
#     print('Fold: ', f + 1)
#     print('#' * 50)

#     # Training Data
#     x_train = X_train[trn_ind]
#     y_trn = y_train[trn_ind]

#     # Validation Data
#     x_val = X_train[val_ind]
#     y_val = y_train[val_ind]

#     # Define start and end epoch for each folds
#     fold_start_epoch = f * EPOCHS
#     fold_end_epoch = EPOCHS * (f + 1)
#     # Create callbacks
#     # checkpoint callback
#     checkpoint_cb = MyModelCheckpoint(
#       os.path.join(run_dir, 'model-{epoch:02d}-{val_loss:.2f}.hdf5'),
#       monitor='val_accuracy', verbose=1)
#     # learning rate callback
#     lr_sched_cb = step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=2)
#     # early stopping callback
#     early = EarlyStopping(monitor='subset_accuracy', min_delta=0.001, patience=15, verbose=1, mode='auto')
#     # csv log callback
#     log_csv = CSVLogger(f'{run_dir}/model_{f + 1}_{now}log.csv', separator=',', append=False)

#     # callbacks list
#     cb = [checkpoint_cb, lr_sched_cb, early, log_csv]

#     model.fit(x_train, y_trn, initial_epoch=fold_start_epoch, epochs=fold_end_epoch,
#               callbacks=cb,
#               batch_size=32,
#               validation_data=(x_val, y_val))

#     elapsed_time = time.time() - start_time
#     print("Elapsed time: {}".format(elapsed_time))


In [63]:
# create AU model
model = create_model()

In [64]:
model

In [58]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_14 (Flatten)        (None, 25088)             0         
                                                                 
 final_au (Dense)            (None, 12)                301068    
                                                                 
Total params: 15,015,756
Trainable params: 301,068
Non-trainable params: 14,714,688
_________________________________________________________________


In [59]:
# load disfa+ dataset
disfa_df = pd.read_csv('AU_image_data.csv')
disfa_df.head()

,Index,Filename,Path,Emotion,AU1,AU2,AU4,AU5,AU6,AU9,AU12,AU15,AU17,AU20,AU25,AU26
0,0,SN001Y_AngerDescribed_TrailNo_1011.jpg,C:\Users\user\Documents\MSc Computer\EmotionDe...,anger,0,0,1,0,0,0,0,0,0,0,0,0
1,1,SN001Y_AngerDescribed_TrailNo_1012.jpg,C:\Users\user\Documents\MSc Computer\EmotionDe...,anger,0,0,2,0,0,0,0,0,0,0,0,0
2,2,SN001Y_AngerDescribed_TrailNo_1013.jpg,C:\Users\user\Documents\MSc Computer\EmotionDe...,anger,0,0,3,0,0,0,0,0,0,0,0,0
3,3,SN001Y_AngerDescribed_TrailNo_1014.jpg,C:\Users\user\Documents\MSc Computer\EmotionDe...,anger,0,0,4,0,0,0,0,0,0,0,0,0
4,4,SN001Y_AngerDescribed_TrailNo_1015.jpg,C:\Users\user\Documents\MSc Computer\EmotionDe...,anger,0,0,4,0,0,0,0,0,0,0,0,0


In [60]:
sample_df = disfa_df.sample(n=950)

In [61]:
X,y = create_dataset(sample_df)

100%|███████████████████████████████████████████████████████████████████████████████| 950/950 [00:07<00:00, 132.73it/s]


In [52]:
X.shape

(950, 224, 224, 3)

In [66]:
y = label_binarizer(y,0.5)

In [67]:
y.shape

(950, 12)

In [68]:
model.fit(X,y,epochs=10)

Epoch 1/10
30/30 [==============================] - 461s 15s/step - loss: 1.5905 - subset_accuracy: 0.3612
Epoch 2/10
30/30 [==============================] - 393s 13s/step - loss: 0.7855 - subset_accuracy: 0.5266
Epoch 3/10
30/30 [==============================] - 410s 14s/step - loss: 0.6350 - subset_accuracy: 0.5259
Epoch 4/10
30/30 [==============================] - 325s 11s/step - loss: 0.5417 - subset_accuracy: 0.5713
Epoch 5/10
30/30 [==============================] - 314s 10s/step - loss: 0.4735 - subset_accuracy: 0.5863
Epoch 6/10
30/30 [==============================] - 286s 10s/step - loss: 0.4205 - subset_accuracy: 0.5993
Epoch 7/10
30/30 [==============================] - 292s 10s/step - loss: 0.3752 - subset_accuracy: 0.6226
Epoch 8/10
30/30 [==============================] - 285s 9s/step - loss: 0.3433 - subset_accuracy: 0.6409
Epoch 9/10
30/30 [==============================] - 288s 10s/step - loss: 0.3115 - subset_accuracy: 0.6519
Epoch 10/10
30/30 [===================